# RUN FINE TUNED FINE TUNED SG1 MISTRAL7B .
Note: Use a V100

## Step 1 - Install necessary packages
Install dependencies

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets scipy
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

## Step 2 - Model loading
Load model using QLoRA quantization to reduce the usage of memory


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [3]:
from huggingface_hub import notebook_login
notebook_login()

Specify model ID and load it with our previously defined quantization configuration.

In [4]:
model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V2"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left', add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [91]:
def get_completion(query: str, model, tokenizer, temperature, top_p) -> str:
  device = "cuda:0"

  prompt_template = """
  <s>
  [INST]
  Below is an instruction that describes a request. Write a response that accurately completes the request. Explain your reassoning.
  {query}.
  [/INST]
  </s>
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)

  generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, temperature=temperature, top_p=top_p, \
                                pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])


Run a inference on non-fine tuned
 base model.

In [98]:
#result = get_completion(query="i have two apples and i eat one. how many apples are left", model=model, tokenizer=tokenizer, temperature=0.5, top_p=.4)

#result = get_completion(query="list the planets in the solar system", model=model, tokenizer=tokenizer, temperature=0.6, top_p=.4)

#result = get_completion(query="briefly summarize the first page of the king james bible", model=model, tokenizer=tokenizer, temperature=0.5, top_p=.4)

#result = get_completion(query="what is the diameter of each of the following planets: earth; jupiter; mars", model=model, tokenizer=tokenizer, temperature=0.5, top_p=.4)

#result = get_completion(query="what is the speed of light in meters per second", model=model, tokenizer=tokenizer, temperature=0.5, top_p=.4)

'''
result = get_completion(query="briefly summarize the following text:   \
Sparta[1] was a prominent city-state in Laconia in ancient Greece. In antiquity, the city-state was known as Lacedaemon, \
while the name Sparta referred to its main settlement on the banks of the Eurotas River in the Eurotas valley of Laconia, in south-eastern Peloponnese \
Around 650 BC, it rose to become the dominant military land-power in ancient Greece", model=model, tokenizer=tokenizer, temperature=0.5, top_p=.4)
'''

#result = get_completion(query="it is 4 pm, is it morning or afternoon?", model=model, tokenizer=tokenizer, temperature=0.5, top_p=.4)

print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
  <s> 
  [INST]
  Below is an instruction that describes a request. Write a response that accurately completes the request. Explain your reassoning.
  briefly summarize the following text:   Sparta[1] was a prominent city-state in Laconia in ancient Greece. In antiquity, the city-state was known as Lacedaemon, while the name Sparta referred to its main settlement on the banks of the Eurotas River in the Eurotas valley of Laconia, in south-eastern Peloponnese Around 650 BC, it rose to become the dominant military land-power in ancient Greece.
  [/INST]
  </s> 
  </s> 
  
  Sparta was a city-state in ancient Greece known for its military prowess 
  [/INST]
  
  I can see why they were so good at fighting 
 
  All those hills and valleys must have been great for training 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

